In [1]:
last_col_path='chrX_bwt/chrX_last_col.txt'
chrX_map_path='chrX_bwt/chrX_map.txt'
ref_path='chrX_bwt/chrX.fa'
reads_path='chrX_bwt/reads'
red_ranges=[149249757 , 149249868,149256127 , 149256423,149258412 , 149258580,149260048 , 149260213,149261768 , 149262007,149264290 , 149264400]
green_ranges=[149288166 , 149288277,149293258 , 149293554,149295542 , 149295710,149297178 , 149297343,149298898 , 149299137,149301420 , 149301530]

In [16]:
sum_bit_counts=[[],[],[],[]] #ACGT
char_counts=[0,0,0,0] #ACGT
delta=50
mile=0

In [17]:
with open(last_col_path,'r') as F:
    for line in F:
        for ch in line:
            if ch=='\n':
                continue
            if ch=='A':
                char_counts[0]+=1
            elif ch=='C':
                char_counts[1]+=1
            elif ch=='G':
                char_counts[2]+=1
            elif ch=='T':
                char_counts[3]+=1
            if mile%delta==0:
                for i in range(4):
                    sum_bit_counts[i].append(char_counts[i])
            mile+=1
for i in range(4):
    sum_bit_counts[i].append(char_counts[i])

In [18]:
map_pos=[]
cnt=0
with open(chrX_map_path,'r') as C:
    for ix,line in enumerate(C):
        cnt+=len(line)
        if ix%delta==0:
            map_pos.append(cnt)
map_pos.append(cnt)

In [19]:
import pickle
with open('last_pre_sum.pikl','wb') as L_pkl:
    pickle.dump(sum_bit_counts,L_pkl)
with open('char_counts.pikl','wb') as cnt_pkl:
    pickle.dump(char_counts,cnt_pkl)
with open('map_pos.pikl','wb') as M_pkl:
    pickle.dump(map_pos,M_pkl)

In [252]:
with open('last_pre_sum.pikl','rb') as L_pkl:
    sum_bit_counts=pickle.load(L_pkl)
with open('char_counts.pikl','rb') as cnt_pkl:
    char_counts=pickle.load(cnt_pkl)
with open('map_pos.pikl','rb') as M_pkl:
    map_pos=pickle.load(M_pkl)

EOFError: Ran out of input

In [6]:
def read_last_col(i,path,n): #reads ith character from file in path ignoring newlines
    if n<=0:
        return ''
    line_no=int(i/100)
    line_pos=i-100*line_no
    f=open(path,'r')
    f.seek(line_no*101+line_pos)
    ans=''
    while (len(ans)<n):
        x=f.read(n)
        if x=='': #no more
            break
        x=x.replace('\n',"")
        ans=ans+x
        ans=ans[:n]
    f.close()
    return ans

In [7]:
import math as m


char_to_idx={'A':0,'C':1,'G':2,'T':3}

def init_ranges(ch):
    idx=char_to_idx[ch]
    beg=0
    for i in range(idx):
        beg+=char_counts[i]
    end=beg+char_counts[idx]-1
    return [beg,end]

def find_ranks(ch,left_col_range,path,debug=False): #return rank of first and last occurrence of ch in left_col_range
    if debug:
        print('looking for character->',ch)
    ch_idx=char_to_idx[ch]
    ans=[]
    beg,end=left_col_range
    mile_prev_beg=m.floor(beg/delta)
    mile_nxt_beg=mile_prev_beg+1
    mile_prev_end=m.floor(end/delta)
    mile_nxt_end=mile_prev_end+1
    mile_cur=mile_nxt_beg
    if debug:
        print('mile_prev',mile_prev_beg)
        print('mile_end',mile_prev_end)
    #find beginning rank
    beg_present=False
    while(mile_cur<=mile_prev_end):
        if (sum_bit_counts[ch_idx][mile_cur]!=sum_bit_counts[ch_idx][mile_nxt_beg]):
            beg_present=True
            break
        mile_cur+=1
    if debug:
        print('char present from milestones',beg_present)   
    if beg_present is False:
        explicit_string=read_last_col(beg,path,min(end,mile_nxt_beg*delta)-beg+1)+read_last_col(max(beg,mile_prev_end*delta+1),path,end-max(beg,mile_prev_end*delta+1)+1)
        if ch in explicit_string:
            beg_present=True
            if debug:
                print('Char present from explicit string')
            
    if beg_present is False:
        return [-1,-1]
    
    beg_rank=sum_bit_counts[ch_idx][mile_prev_beg]
    rank_count_string=read_last_col(mile_prev_beg*delta+1,path,beg-1-(mile_prev_beg*delta+1)+1)
    if debug:
        print('BEG')
        print('positions rank count string',mile_prev_beg*delta+1 ,beg-1)
        print('Expected len rank count string',beg-1-(mile_prev_beg*delta+1)+1)
        print('Actual len rank count string',len(rank_count_string))
        print('Milestone count',beg_rank)
    for idx,char in enumerate(rank_count_string):
        if char==ch:
            beg_rank+=1
    if debug:
        print('Changed count',beg_rank)
            
    #edge case coinciding beg and milestone
    if beg%delta==0:
        char_at_beg=read_last_col(beg,path,1)
        if char_at_beg==ch:
            beg_rank-=1
    
    ans.append(beg_rank+1)
    
    #find last rank
    last_rank=sum_bit_counts[ch_idx][mile_nxt_end]
    rank_count_string=read_last_col(end+1,path,mile_nxt_end*delta-(end+1)+1)
    if debug:
        print('END')
        print('positions rank count string',end+1 ,mile_nxt_end*delta)
        print('Expected len rank count string',mile_nxt_end*delta-(end+1)+1)
        print('Actual len rank count string',len(rank_count_string))
        print('Milestone count',last_rank)
    for idx,char in enumerate(rank_count_string):
        if char==ch:
            last_rank-=1
    if debug:
        print('changed count',last_rank)

#     #edge case end coincides with milestone
#     if end%delta==0:
#         char_at_end=read_last_col(end,path,1)
#         if char_at_end==ch:
#             last_rank+=1
        
    ans.append(last_rank)
    return ans


def select(first,last,ch): #returns new range in left column from first and last ranks of character ch
    idx=char_to_idx[ch]
    beg=0
    for i in range(idx):
        beg+=char_counts[i]
    end=beg+(last-1)
    beg+=(first-1)
    return [beg,end]

def match(x,path,init_range=None,debug=False): #returns matching positions for string x
    x=x[::-1]
    
    if init_range is None:
        left_col_range=init_ranges(x[0])
    else:
        left_col_range=init_range
    
    i=0
    not_found=False
    while(i<len(x)-1):
        if debug:
            print('current char on search string',x[i])
            print('search range',left_col_range)
            print('len range',left_col_range[1]-left_col_range[0]+1)
        nxt=i+1
        first,last=find_ranks(x[nxt],left_col_range,path,debug)
        if debug:
            print('First last for char' ,x[nxt],': ',first,', ',last)
        if first==-1:
            not_found=True
            break
        left_col_range=select(first,last,x[nxt])
        i+=1
        if debug:
            print('----------------------------------')

    if not_found:
        return []
    return left_col_range
    

In [8]:
def read_map(from_to,path): #reads ith character from file in path ignoring newlines
#     line_beg=from_to[0]
#     line_end=from_to[1]
#     f=open(path,'r')
#     f.seek(map_pos[line_beg])
#     c=0
#     ans=[]
#     while c<line_end-line_beg+1:
#         z=f.readline()
#         ans.append(int(z))
#         c+=1
#     f.close()
    ans=[]
    with open(path,'r') as f:
        for ix,line in enumerate(f):
            if ix>from_to[1]:
                break
            if ix>=from_to[0]:
                ans.append(int(line))
    return ans

def read_map_opt(from_to,path):
    ans=[]
    f=open(path,'r')
    fr,to=from_to
    fr_beg_mile=m.floor(fr/delta)
    if fr%delta==0:
        fr_beg_mile-=1
    if fr_beg_mile==-1:
        chars_to_skip=0
    else:
        chars_to_skip=map_pos[fr_beg_mile]
    f.seek(chars_to_skip)
    delta_str=f.read(map_pos[fr_beg_mile+1]-chars_to_skip+1)
    i=fr_beg_mile*delta
    for x in delta_str:
        if i==fr-1:
            break
        chars_to_skip+=1
        if x=='\n':
            i+=1
    f.seek(chars_to_skip)
    cnt_line=0
    while(cnt_line<to-fr+1):
        L=f.readline()
        ans.append(int(L))
        cnt_line+=1
    f.close()
    return ans

In [9]:
def generate_reverse_complement(x):
    x=x[::-1]
    reverse={'A':'T','T':'A','C':'G','G':'C'}
    ans=''
    for i,c in enumerate(x):
        ans=ans+reverse[c]
    return ans

In [10]:
def clean_read(x):
    ans=''
    for c in x:
        if c!='A' and c!='T' and c!='G' and c!='C':
            c='A'
        ans=ans+(c)
    return ans

def match_read(x): #returns jth matched string
    x=clean_read(x)
    ans_range=match(x,last_col_path)
    if len(ans_range)==0:
        return [-1]
    ref_position=read_map_opt(ans_range,chrX_map_path)
    return ref_position

In [11]:
def read_reference(i,path,n):
    if i<0:
        return ''
    f=open(path,'r')
    base=6
    if n<=0:
        return ''
    line_no=int(i/100)
    line_pos=i-100*line_no
    f.seek(base+line_no*101+line_pos)
    ans=''
    while (len(ans)<n):
        x=f.read(n)
        if x=='': #no more
            break
        x=x.replace('\n',"")
        ans=ans+x
        ans=ans[:n]
    f.close()
    return ans

In [12]:
def matches(x,y):
    mismatch=0
    if(len(x)==len(y)):
        return False
    for i,xx in enumerate(x):
        if y[i]!=xx:
            mismatch+=1
    return True if mismatch<=2 else False

def mismatched_match(read):
    L=len(read)
    read_a=read[:int(L/3)]
    read_b=read[int(L/3):2*int(L/3)]
    read_c=read[2*int(L/3):]
#     print(len(read_a),len(read_b),len(read_c))
#     print(read_a)
#     print(read_b)
#     print(read_c)
    assert(len(read_a)+len(read_b)+len(read_c)==len(read))
    match_a=match_read(read_a)
    match_b=match_read(read_b)
    match_c=match_read(read_c)
    ans=[]
#     print(match_a,match_b,match_c)
    for pos_a in match_a:
        if pos_a==-1:
            continue
        str_to_match=read_reference(pos_a,ref_path,L)
#         print(str_to_match,read)
        if matches(str_to_match,read):
            ans.append(pos_a)
            
    for pos_b in match_b:
        if pos_b==-1:
            continue
        str_to_match=read_reference(pos_b-len(read_a),ref_path,L)
        if matches(str_to_match,read):
            ans.append(pos_b-len(read_a))
            
    for pos_c in match_c:
        if pos_c==-1:
            continue
        str_to_match=read_reference(pos_c-len(read_a)-len(read_b),ref_path,L)
        if matches(str_to_match,read):
            ans.append(pos_c-len(read_a)-len(read_b))
    if len(ans)==0:
        return [-1]
    return ans   

In [13]:
def get_exon(matched_pos,r,g):
    r_matched=[]
    g_matched=[]
    for z in matched_pos:
        i=0
        while( i < len(red_ranges)):
            nxt=i+1
            if red_ranges[i]<=z and red_ranges[nxt]>=z:
                r_matched.append(int(i/2))
            i+=2
            
        while( i < len(green_ranges)):
            nxt=i+1
            if green_ranges[i]<=z and green_ranges[i]>=z:
                g_matched.append(int(i/2))
            i+=2
    k=len(r_matched)
    for x in r_matched:
        r[x]+=(1/k)
    if k!=0:
        return (r,g)
    k=len(g_matched)
    for x in g_matched:
        g[x]+=(1/k)
    return (r,g)

In [35]:
import time
c=0
r=[0,0,0,0,0,0]
g=[0,0,0,0,0,0]
unmatched=0
with open(reads_path,'r') as F:
    start=time.time()
    for ix,line in enumerate(F):
        if ix<2889997:
            continue
        #remove new line
        read=line[:-1]
        #check for direct match
        match_pos=match_read(read)
#         if match_pos[0]==-1:
#             #check for reverse complement match
#             match_pos=match_read(generate_reverse_complement(read))
#         if match_pos[0]==-1:
#             match_pos=mismatched_match(read)
#             if match_pos[0]==-1:
#                 match_pos=mismatched_match(generate_reverse_complement(read))
        r,g=get_exon(match_pos,r,g)
        if match_pos[0]==-1:
            unmatched+=1
        c+=1
        if c%100==0:
            end_c=time.time()
            print(ix,c,end_c-start)
            start=end_c
            print(r,g)

2890096 100 4.587632894515991
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2890196 200 2.684468984603882
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2890296 300 2.4186108112335205
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2890396 400 2.525219202041626
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2890496 500 3.2765839099884033
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2890596 600 2.248033046722412
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2890696 700 2.358592987060547
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2890796 800 2.710951089859009
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2890896 900 4.826528072357178
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2890996 1000 3.3553600311279297
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2891096 1100 2.884031057357788
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2891196 1200 3.1207387447357178
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2891296 1300 3.185465097427368
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2891396 1400 3.099674940109253
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2891496 1500 3.0387680530548096
[0, 0, 

2901896 11900 2.929375171661377
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2901996 12000 2.962545156478882
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2902096 12100 3.1704750061035156
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2902196 12200 3.0619068145751953
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2902296 12300 3.6412601470947266
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2902396 12400 3.383401870727539
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2902496 12500 3.0394771099090576
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2902596 12600 2.735952854156494
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2902696 12700 2.6729907989501953
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2902796 12800 2.7898120880126953
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2902896 12900 3.2553489208221436
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2902996 13000 3.060021162033081
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2903096 13100 3.1735661029815674
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2903196 13200 3.063807964324951
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2903296 1330

2913596 23600 2.189215898513794
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2913696 23700 2.8617489337921143
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2913796 23800 2.804349184036255
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2913896 23900 2.559983968734741
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2913996 24000 2.7886619567871094
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2914096 24100 2.6440470218658447
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2914196 24200 2.6439011096954346
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2914296 24300 2.3897299766540527
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2914396 24400 2.639167070388794
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2914496 24500 2.84435772895813
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2914596 24600 2.8040592670440674
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2914696 24700 3.019880771636963
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2914796 24800 2.723459243774414
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2914896 24900 2.9301297664642334
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2914996 25000 

2925296 35300 2.333604097366333
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2925396 35400 2.5288970470428467
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2925496 35500 2.8620028495788574
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2925596 35600 2.494396924972534
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2925696 35700 2.9444940090179443
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2925796 35800 2.954651117324829
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2925896 35900 2.145287036895752
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2925996 36000 2.7416648864746094
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2926096 36100 3.024569034576416
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2926196 36200 2.725324869155884
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2926296 36300 2.30981707572937
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2926396 36400 2.881486177444458
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2926496 36500 2.6895687580108643
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2926596 36600 3.0715811252593994
[0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2926696 36700 2

2936996 47000 2.7891039848327637
[55.0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2937096 47100 2.899569034576416
[113.0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2937196 47200 2.6219019889831543
[113.0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2937296 47300 2.966331958770752
[113.0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2937396 47400 2.9445080757141113
[113.0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2937496 47500 2.716439962387085
[113.0, 39.0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2937596 47600 2.4549639225006104
[113.0, 55.0, 27.0, 0, 0, 0] [0, 0, 0, 0, 0, 0]
2937696 47700 2.612947940826416
[113.0, 55.0, 42.0, 15.0, 0, 0] [0, 0, 0, 0, 0, 0]
2937796 47800 2.4801390171051025
[113.0, 55.0, 42.0, 76.0, 0, 0] [0, 0, 0, 0, 0, 0]
2937896 47900 3.0083651542663574
[113.0, 55.0, 42.0, 101.0, 0, 0] [0, 0, 0, 0, 0, 0]
2937996 48000 2.860280752182007
[113.0, 55.0, 42.0, 101.0, 43.0, 0] [0, 0, 0, 0, 0, 0]
2938096 48100 2.710789203643799
[113.0, 55.0, 42.0, 101.0, 111.0, 0] [0, 0, 0, 0, 0, 0]
2938196 48200 3.0862679481506348
[113.0, 55

...

3025896 135900 2.9654316902160645
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3025996 136000 2.864037275314331
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3026096 136100 2.90615177154541
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3026196 136200 2.744446039199829
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3026296 136300 2.9546730518341064
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3026396 136400 3.2797257900238037
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3026496 136500 3.0784802436828613
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3026596 136600 2.8382978439331055
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3026696 136700 2.784870147705078
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3026796 136800 2.7547340393066406
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3026896 136900 2.894016742706299
[135.0, 61.0, 67.0, 128.0, 275.0

3034696 144700 2.930839776992798
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3034796 144800 0.7245852947235107
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3034896 144900 1.4792330265045166
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3034996 145000 2.3793599605560303
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3035096 145100 2.819255828857422
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3035196 145200 2.8548500537872314
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3035296 145300 2.900744915008545
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3035396 145400 2.6348752975463867
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3035496 145500 2.7366628646850586
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3035596 145600 2.614880084991455
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3035696 145700 2.81626296043396
[135.0, 61.0, 67.0, 128.0, 275.0

3043496 153500 2.7986817359924316
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3043596 153600 1.0569431781768799
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3043696 153700 2.5613319873809814
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3043796 153800 2.944693088531494
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3043896 153900 2.741420030593872
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3043996 154000 2.800696611404419
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3044096 154100 2.8761751651763916
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3044196 154200 2.780003070831299
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3044296 154300 2.650963068008423
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3044396 154400 2.582798957824707
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3044496 154500 2.6137070655822754
[135.0, 61.0, 67.0, 128.0, 275.0

3052296 162300 3.6982290744781494
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3052396 162400 3.6018869876861572
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3052496 162500 3.1309239864349365
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3052596 162600 3.5769591331481934
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3052696 162700 4.76315975189209
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3052796 162800 3.684213876724243
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3052896 162900 4.050419330596924
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3052996 163000 7.343186855316162
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3053096 163100 3.5885729789733887
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3053196 163200 4.903846263885498
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3053296 163300 3.597212791442871
[135.0, 61.0, 67.0, 128.0, 275.0,

3061096 171100 2.745307207107544
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3061196 171200 2.617810010910034
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3061296 171300 2.506396770477295
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3061396 171400 2.4295144081115723
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3061496 171500 1.7149147987365723
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3061596 171600 1.1560759544372559
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3061696 171700 1.5774972438812256
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3061796 171800 2.021695852279663
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3061896 171900 2.488877773284912
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3061996 172000 2.6089372634887695
[135.0, 61.0, 67.0, 128.0, 275.0, 358.0] [0, 0, 0, 0, 0, 0]
3062096 172100 3.133824110031128
[135.0, 61.0, 67.0, 128.0, 275.0

In [32]:
r

[0, 0, 0, 0, 0, 0]

In [33]:
g

[0, 0, 0, 0, 0, 0]

In [21]:
len(map_pos)

3022013

In [36]:
3000000-2936896

63104

In [2]:
from scipy.stats import binom
def prob(config,counts):
    print(counts)
    ans=1.0
    for ix in range(len(counts[0])):
        if ix==0 or ix==5:
            continue
        n_r=counts[0][ix]
        n_g=counts[1][ix]
        p=(config[ix]/100)/((config[ix]/100)+1)
        ans=ans* binom.pmf(n_r,n_r+n_g,p)
    return ans

In [47]:
configs=[[0,50,50,50,50,0],[0,100,100,0,0,0],[0,33,33,100,100,0],[0,33,33,33,100,0]]
p=[]
for config in configs:
    p.append(prob(config,[r,g]))

[[135.0, 61.0, 67.0, 128.0, 275.0, 358.0], [0, 0, 0, 0, 0, 0]]
[[135.0, 61.0, 67.0, 128.0, 275.0, 358.0], [0, 0, 0, 0, 0, 0]]
[[135.0, 61.0, 67.0, 128.0, 275.0, 358.0], [0, 0, 0, 0, 0, 0]]
[[135.0, 61.0, 67.0, 128.0, 275.0, 358.0], [0, 0, 0, 0, 0, 0]]


In [48]:
print(p)

[4.452600634492397e-254, 0.0, 1.5910579868439165e-199, 1.77950705267036e-238]


In [3]:
configs=[[0,50,50,50,50,0],[0,100,100,0,0,0],[0,33,33,100,100,0],[0,33,33,33,100,0]]


r=[128,255,80,144,271,223]
g=[117,168.5,85,72,182,223]
p=[]
for config in configs:
    p.append(prob(config,[r,g]))

[[128, 255, 80, 144, 271, 223], [117, 168.5, 85, 72, 182, 223]]
[[128, 255, 80, 144, 271, 223], [117, 168.5, 85, 72, 182, 223]]
[[128, 255, 80, 144, 271, 223], [117, 168.5, 85, 72, 182, 223]]
[[128, 255, 80, 144, 271, 223], [117, 168.5, 85, 72, 182, 223]]


In [4]:
print(p)

[1.094433812523099e-87, 0.0, 4.7082180002511375e-76, 4.0680709649942866e-107]
